# 2: Data Input/Output and Point Process Statistics

## Representing neural data in digital computers

Computational neuroscience often involves a lot of data. In this lesson, you'll learn about:

- different kinds of data: how they are represented digitally
- how to read data from the disk into your program
- how to store data from your program to the disk

We'll consider three kinds of data:

- [time series](#Time-series-data)
- [point processes](#Point-processes)
- [structured records](#Structured-records)

# Time series data

Typically represented as an **array** of measurements: 

$$\mathbf{x} = \{x_0, x_1, \ldots, x_N\}$$

Can be **multichannel** if more than one measurement taken at a time. Each time point is now a **vector** ($\vec{x}$):

$$\mathbf{X} = \{\vec{x}_0, \vec{x}_1, \ldots, \vec{x}_N\}$$

Multichannel time series are represented as two-dimensional arrays. One dimension correponds to time and the other to the component of the measurement vector.

Note that the "channels" can be repeated **trials** rather than simultaneous measurements.

## Time series in Python

We use `numpy` arrays to store single- and multichannel time series in Python. Let's look at an example using some Gaussian white noise.

Gaussian noise is drawn from a normal distribution, and it's called white noise because it has equal power at all frequencies.

In [ ]:
# load matplotlib inline mode
%matplotlib inline

# import some useful libraries
import sys
import os
import numpy as np                # numerical analysis linear algebra
import matplotlib.pyplot as plt   # plotting
sys.path.insert(0,"/project/psyc5270-cdm8j/comp-neurosci")
from comp_neurosci_uva import signal, data

In [ ]:
np.random.seed(1)                  # set random seed so we all get the same results
x = np.random.randn(1000)          # generate 100 random WN samples

plt.plot(x)

You can determine the number of elements in a 1D array using the `len()` function:

In [ ]:
print("The number of points in x is:", len(x))

**Q**: Do you recall from the last exercise how to access subsets of a numpy array? In the code cell below, write an expression to evaluate the mean of the first 100 samples of `x`:

Numpy (and Python in general) supports **negative indexing**, which means that negative indices are interpreted as referencing elements from the **end** of the array. The following expression gives us the mean of the last 100 elements.

In [ ]:
np.mean(x[-100:1000])
# you can leave out the second index in the slice if it refers to the end of the array, so this is equivalent:
np.mean(x[-100:])

## Multichannel time series

For multichannel data, the array has two dimensions. There is a (weak) convention that the first dimension of the array represents time. That means each column represents a separate channel.

Here is an example of a 3-channel array, again using Gaussian white noise. I've added some correlations between the channels to make things interesting.

In [ ]:
y = np.random.multivariate_normal(mean=[0, 0, 0], cov=[[1.0, 0.2, 0.0],[0.2, 1.0, 0.1], [0.0, 0.1, 1.0]], size=1000)
plt.plot(y);

Notice how the plot now has three different-colored traces? Matplotlib assumes that time is the first dimension when you give it an array to plot.

The size along each dimension of the array is called its **shape**. You can get the shape (and therefore the dimension of an array) using the `.shape` attribute:

In [ ]:
print("The shape of y is:", y.shape)
# note that len returns the number of elements along the first dimension
print("The number of time points in y is:", len(y))
print("The total size of y is:", y.size)

For multichannel data, we need two indices or slices to access values in the array.

In [ ]:
# the first time point in the first channel. Note the comma.
y[0, 0]

In [ ]:
# use `:` to indicate all the values along one dimension. This gives all the values for the first channel
plt.plot(y[:, 0])

In [ ]:
# to get all the channels at a time point:
print("y_0 =", y[0, :])
# you can leave out the trailing indices
print("y_0 =", y[0])

**Q**: Calculate and plot the time series defined by the mean of all three channels

In [ ]:
## Enter code here

## Input/Output 

Presumably you'd like to look at more than just randomly generated noise. So how do you get data from a recording into your program?

Usually, data are stored long-term on your computer's drive or in the cloud. There are advantage to both approaches, which we'll discuss later. For now, we're going to retrieve some files stored in a shared directory. Once we do this, we'll see how to load the data from these files.

### I/O for Time Series

Unfortunately, there is no agreed-upon standard for storing time series data, so you'll often have to do some sleuthing.

There are three major kinds of storage formats: text, binary, and custom.

### Text

One way of storing numbers is how you would write them (i.e., as **text**). 

When reading a text file, the main thing you need to know is how the elements are separated. 

For single-channel data, usually each number goes on its own line. 

For multi-channel data, there will be multiple numbers per line, typically separated by white space (tabs and/or spaces) or by commas.

When you're storing data in text format, you also need to be mindful of the precision. 

#### Reading text files

One advantage of storing data as text is that it's human-readable. However, this isn't as much of an advantage as you might think. 

To see an example of an extracellular recording in text format, switch to the main Juptyer tab in your browser, navigate to the `data/io-examples` folder, and click on one of the files that ends in `.txt`

Numpy can easily load single- and multi-channel data from text files using the `loadtxt` command:

In [ ]:
path = os.path.join(data.data_path, "io-examples", "st11_1_2_A8.txt")
d = np.loadtxt(path)
plt.plot(d)

### Binary

Storing numbers as text is very inefficient. Let's see why.

In [ ]:
## text is read into Python first as a string:
s = open(path, "r").readline().strip()
print("the number as text:", s)
print("size of the text (in bytes):", len(s))

## to use it as a number, python has to parse the text
f = float(s)
print("the number as a float:", f)
print("size of a float (in bytes):", d.dtype.itemsize)

#### Reading binary data

Not only does text-formatted data take up a lot more space, it also requires additional work for Python to translate into a numerical representation that it can do math on (i.e., floats and ints).

This inefficiency becomes a consideration for large datasets. Thus, we often want to store the data on disk in a binary format, i.e., the same format as it would be in memory. When you load binary data from disk, it's stored directly in an array, without any need for parsing.

Another reason to store data in a binary format is that it allows your program to directly map the file into memory as a **memory map**. Instead of reading the entire file into memory, Python only reads the parts of the file that you actually use, when you want to use them.

In [ ]:
import os   # standard library module used to construct paths

path = os.path.join(data.data_path, "io-examples", "st11_1_2_A8.dat")
d = np.memmap(path, mode="r", dtype='d')
plt.plot(d)

## Structured formats

Text and raw binary formats both have shortcomings and tradeoffs.

A shortcoming they both have in common is that it can be difficult to store metadata.

Without metadata, it may be hard to know how to interpret the contents of the file.

Some critical metadata we need for time series include:

- sampling rate
- dimensions of the array
- ordering of the array (i.e., time first or last) and what's in each channel
- measurement units

**Q**: What other metadata do you think are important for time series data? Write a couple of ideas in the cell below.

Structured data formats can be text- or binary-based.

Some formats are used widely and are well-documented, like [Javascript Object Notation](http://json.org) or [HDF5](https://support.hdfgroup.org/HDF5/). These formats are likely to have well-supported Python packages for I/O.
    
Other formats are more obscure or proprietary, like Axon Binary Format (ABF) or [Elan](http://elan.lyon.inserm.fr/). It may be difficult to find Python support to read these files, though the situation is improving thanks to projects like [Neo IO](https://neo.readthedocs.io/).

# Point processes

Typically represented as an ordered sequence of times in some interval from 0 to $T$:

$$\{0 \leq t_0 < t_1 < \ldots < t_N \leq T\}$$

In contrast to time series, there is not a fixed relationship between the number of events and the duration of the analysis interval.

## Point process data in Python

Point processes are also typically stored in `numpy` arrays, but the elements of the array are event times, not measurements.

Because point processes vary in the number of events, multi-channel point-processes are represented by **lists of arrays**, not by 2D arrays.

Let's look at some examples:

In [ ]:
from comp_neurosci_uva import pprox
resp = data.load_pprox("starling", "pprox", "st11_1_2_1")
resp_A8 = pprox.select_stimulus(resp, "A8")
resp_A8

The variable `resp_A8` refers to a Python **list**. Lists are like arrays, but they can store heterogeneous data types. The syntax for accessing elements and slices is the same.

**Q:** Using what you know from previous exercises, complete the following code cell to print out some information about the data: 

In [ ]:
print("The number of trials is:", ???)
print("The number of events in trial 0 is:", ???)
print("The time of the first event in trial 2 is:", ???)

## I/O for Point Processes

Just as there is no agreed-upon standard for storing time series data, there is also no standard format for point-process data.

Because point process data tend to be smaller than time series, text formats are more common than binary.

A very simple text format is to put each trial (or channel) on a separate line and separate the events on each line with a space. Take a look at `data/io-examples/st_11_2_1_A8.txt` for an example.

The [PySpike](http://mariomulansky.github.io/PySpike/) library has a function for loading data from such files, but we're going to write our own so that we can learn a bit about basic I/O in Python and looping.

In [ ]:
# create a list where we will store our trials
trials = []
# open the file for reading
fp = open(os.path.join(data.data_path, "io-examples", "st11_1_2_1_A8.txt"), mode="r")
# loop through the lines of the file with a for statement
for line in fp:
    # read the line into an array
    arr = np.fromstring(line, sep=" ")
    # append the array to our list
    trials.append(arr)

If you learned how to program in Java or C or another low-level programming language, take a moment to appreciate how simple this task is in Python.

### Checking our work

**Q:** The `trials` list we created in the last code cell should be the same as `resp_A8`. In the cell below, complete **three assert** statements to check that this is true. I've provided you with one to get started. If you complete your task correctly, the cell will not emit any errors.

In [ ]:
assert len(trials) == len(resp_A8), "The number of trials is not the same"
assert True == False, "The total number of events is not the same"

### Point process metadata

As with time series data, it's important to keep track of metadata. Here are some important metadata that need to be associated with point process files:

- type of event (e.g., spike, behavioral action, stimulus start/stop)
- number of channels
- unit scaling (e.g., milliseconds or seconds?)
- start time
- other experimental variables

## Structured Records

In both point process and time series data, the elements of the arrays have been **homogeneous** (i.e., all the same type). What if that's not the case?

The third (and final) kind of data we'll consider today consists of **records**. Each record in turn comprises **fields**, which may have different types.

This kind of data is also called **tabular data**. If you're coming from the R world, you might think of this kind of data as a `data.frame`.

It's common to encounter structured records when you have independent observations; for example, from different neurons or animals or populations. The fields in each record might include:

- a unique identifier for the observation
- group identifiers (e.g., cell, animal, population)
- independent variables (e.g., sex, treatment, age)
- dependent variable(s)

## Structured records in Python

Python used to lag pretty badly behind R for handling this kind of data, but we now have [pandas](http://pandas.pydata.org/), which is beginning to approach `numpy` in popularity and maturity.

As with numpy, there is a convention for importing pandas: 

```python
import pandas as pd
```

For a detailed introduction to pandas, take a look at [Chapter 3](https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html) of the Python Data Science Handbook.

There are two main concepts to understand in using pandas: `Series` and `DataFrames`.

A `Series` is essentially a column of a table. Like a numpy array, all the elements of a series are the same type. Unlike a numpy array, the indices of a `Series` do not have to be sequential integers, but can be any label you like.

For example, here's a `Series` that might represent the ages of several subjects:

In [ ]:
import pandas as pd
ages = pd.Series([391, 442, 183], index=['st11', 'st22', 'st231'])
ages

You can access the elements of a `Series` using the standard Python bracket syntax:

In [ ]:
ages['st11']

A `DataFrame` is a collection of `Series`, i.e. a table of columns. Here's how we might represent the ages and sexes of a set of subjects:

In [ ]:
sex = pd.Series(['M', 'F', 'M'], index=['st11', 'st22', 'st231'])
subjects = pd.DataFrame({'age': ages, 'sex': sex})
subjects

Notice how we now have a table, which Jupyter renders nicely for us with the row and column indices are indicated in bold.

The bracket syntax for `DataFrames` accesses **columns**. It's important to remember that this is different from numpy arrays, where a single index gives you a row.

In [ ]:
subjects['age']

To index by row and column, you have to use the `loc` attribute:

In [ ]:
subjects.loc['st11', 'age']

Equivalently, you can use `iloc` and the numerical indices:

In [ ]:
subjects.iloc[0, 0]

## I/O for Structured Records

Structured record data is usually stored on disk in text-based formats. This is because human readability is often quite important. There are two very common formats:

- In _comma-separated-value_ files, each record is on a separate line, and fields are separated by commas.
- In _whitespace-delimited-value_ files, each record is on a separate line, and fields are separated by white space (tabs or spaces)

In both kinds of files, it's common that the first line of the file is a header giving the name for each column.

Take a look at `data/stimuli/motifs.csv` for an example of a comma-delimited file.

One really good reason to use pandas is that it provides some nice I/O functions for these kinds of files. It's trivial to load tabular data into Python:

In [ ]:
motifs = pd.read_csv(os.path.join(data.data_path, "starling", "stimuli", "motifs.csv"))
motifs

You can tell pandas that certain columns should be used as indices.

This allows you to select a subset of rows using the `loc` syntax. For example, to see all the rows where `song` is equal to `A8`:

In [ ]:
motifs = motifs.set_index(['song'])
motifs.loc['A8']

### Exercise

Write code in the cell below to compute the following:

- the number of different songs
- the number of motifs for each song
- the average motif duration in each song

## Spike Train Statistics

As we've seen from previous examples, much of the data generated by the brain consists of spikes (i.e., action potentials). We all know that neurons spike when they are excited, but what does this mean quantitatively?

In this part of the lesson, we'll dive into a fairly simple but foundational model that attempts to quantify spiking as the function of an underlying **rate**.

Our goals are to:

- understand homogeneous and inhomogeneous Poisson process models
- be able to estimate the latent rate variable in Poisson models

Follow along in the notebook during the lecture, and then work on the cells marked **Q** with help from your instructor. Submit the completed notebook to Collab.

### More point process math

Recall that a point process is an (ordered) sequence of event times:

$$X = \{t_0, t_1, \ldots, t_{N-1}\}$$

We can represent this as a function by making each spike a [Dirac delta function](https://en.wikipedia.org/wiki/Dirac_delta_function):

$$\rho(t) = \sum_{i=0}^{N-1} \delta(t - t_i)$$

![spike_train](images/l5_spike_train.png "spike train delta function")

Because the area under each delta function is 1, this allows us to count spikes or calculate any continuous function of a spike train through integration.

For example, the **rate** is defined as the number of spikes $N$ that occurred in some interval divided by the duration of the interval, $T$:

$$R = \frac{N}{T} = \frac{1}{T} \int_{0}^{T} d\tau\; \rho(\tau)$$

![spike_train_rate](images/l5_rate_integral.png "spike train rate integral")

### Spiking as a random variable

Neural responses are **stochastic**. Even under "identical" conditions, spike trains will vary from trial to trial.

In other words, the response $\rho(t)$ is a **random variable**. The probability of observing a particular response is given by a **distribution**, $p(\rho(t))$.

We can also represent the stimulus $\vec{s}(t)$ as a random variable with a distribution $p(\vec{s}(t))$, even if it's under experimental control.

### Review of probability

Leaving out the notation indicating that both $\rho$ and $\vec{s}$ are functions of time, the following distributions are of interest:

The joint distribution $p(\rho, \vec{s})$ is the probability of $\rho$ and $\vec{s}$ occurring together. This is likely to be a very small number because the space of $\vec{s}$ and $\rho$ are potentially quite large.

Thus, we are often more interested in the conditional probability $p(\rho|\vec{s})$, which is the probability of observing $\rho$ when $\vec{s}$ is presented.

The marginal probability $p(\rho)$ indicates the probability of observing $\rho$ irrespective of which stimulus was presented.

There are two mathematical identities that allow us to convert between these distributions. 

#### Conditional Probability

$$p(\rho,\vec{s}) = p(\rho|\vec{s})p(\vec{s}) = p(\vec{s}|\rho)p(\rho)$$

If $p(\rho|\vec{s}) = p(\rho)$, then $\rho$ and $\vec{s}$ are **independent**. Independence means $p(\rho)$ is the same regardless of what $\vec{s}$ is. This means that for independent variables, the joint distribution is simply the product of the marginal distributions.

$$p(\rho,\vec{s}) = p(\rho)p(\vec{s})$$

#### Marginal Probability

We can convert a joint probability distribution to a marginal probability distribution by summing (integrating) the probabilities of one of the variables.

$$p(\rho) = \int_S d\vec{s}\; p(\rho, \vec{s}) = \int_S d\vec{s}\; p(\rho|\vec{s}) p(\vec{s})$$

The $S$ under the integral means that we are summing up $p(\rho,\vec{s})$ over every possible stimulus. Remember that every probability density function has to integrate to 1.0:

$$\int_S d\vec{s}\; p(\vec{s}) = 1$$

### Spike-train statistics

Let's apply these concepts to spike trains:

The probability of of a sequence of $N$ spikes $X = \{t_0,\ldots,t_{N-1}\}$ is the joint probability density of all the individual spikes: 

$$p(t_0, t_1, \ldots, t_{N-1})$$

If the spikes are independent, then this joint distribution is simply the product of the distributions for each spike:

$$p(t_0, \ldots, t_{N-1}) = \prod_{i=0}^{N-1}p(t_i)$$

When each spike is independent of every other spike, we have a **Poisson process**.

### Homogeneous Poisson Processes

If $t_i$ is independent of all the other spikes, what does it depend on?

In the simplest case, $p(t_i)$ is a constant: the probability of observing a spike at any given time is a single number, which corresponds to the **rate** of spiking, $R$.

If the rate is constant, the Poisson process is **homogeneous**. In this case, in an interval $(t_i, t_i + \Delta)$, we would expect to observe $\lambda = R\Delta$ events. The distribution of the number of events we actually observe, $n$, is given by the Poisson distribution:

$$p(n|\lambda) = \frac{\lambda^n}{n!}\exp(-\lambda)$$

The parameter $\lambda$ is often called the **intensity** of the process.

Let's explore some properties of the Poisson distribution in Python. Here's a graph of the distribution from wikipedia:

![poisson_distro](https://upload.wikimedia.org/wikipedia/commons/1/16/Poisson_pmf.svg)

In [ ]:
# load matplotlib inline mode
%matplotlib widget

# import some useful libraries
import numpy as np                # numerical analysis linear algebra
# notebook modules
import IPython
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import display
import ipywidgets as widgets

# import the poisson distro
from comp_neurosci_uva import dists

In [ ]:
# first, let's define our *support*: the values over which we want to evalute p(n):
supp = np.arange(0, 20)

# next, we *instantiate* the distribution object with our parameter lambda
dist = dists.poisson(1.0)

# you can get the probability of any value in the distribution with .pmf. Note that we have to use
# pmf (probability mass function) rather than pdf.
print("p(5|lambda=1) =", dist.pmf(5))

# we can also evaluate the distribution over a vector of numbers
prob = dist.pmf(supp)

# and plot the distribution with plt.plot
plt.plot(supp, prob, lw=3);

**Q**: In the cell below, try to evaluate the `prob` distribution for negative or non-integral numbers. Given the definition of the Poisson distribution, why is this the case?

**Q** Recall that $\lambda = R\Delta$. If $R = 1$ Hz and you steadily reduce $\Delta$ from 1.0 s to 1.0 microseconds, what is the probability of observing one spike in that interval? Write a *for loop* to evaluate and plot this. Does the result make sense? What is the probability of a spike occurring at some *exact* time?

In [ ]:
Delta = [1e0, 5e-1, 1e-1, 5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5, 5e-6, 1e-6]

In addition to plotting the probability distribution, Python can generate random samples (i.e, **draw**) from the Poisson distribution, too.

In [ ]:
# note: 'lambda' is a reserved symbol in python
lam = 1.0
dist = dists.poisson(lam)
# rvs stands for random value(s)
n = dist.rvs(1000)

**Q** In the cell below, calculate the sample mean and standard deviation for n, then try with a few other values of lambda.

In [ ]:
print("The sample mean when lambda=", lam, "is", np.mean(n))

**Q**: Based on your exploratory analysis, fill out the following identities:

- Mean: $\mu =$
- Standard deviation: $\sigma =$
- Fano factor $\sigma^2/\mu =$

### From Poisson distribution to Poisson process

How can we generate a series of spike times from the Poisson distribution? The trick is to divide your response interval up into a set of smaller intervals (or **bins**) such that the probability of observing more than one spike in a single bin is very small, then draw from $p(n|\lambda)$ for each bin.

In [ ]:
np.random.seed(1)
T     = 100     # s
rate  = 4.0     # Hz
Delta = 0.005   # s
dist   = dists.poisson(rate * Delta)
hom_spikes = dist.rvs(int(T / Delta))
bins   = np.arange(0, T, Delta)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 3))
axes.plot(bins, hom_spikes)

The plot of the `spikes` array should only show `0` and `1` values. If it doesn't, try adjusting the `Delta` variable in the code cell above. What direction does it need to change to fix the problem? Why?

### Spike arrays and spike times

You can think of the `spikes` array as a sort of time series representation of the point process. 

To get the actual spike times, we need to find the bins where there is a spike and then look up the times in the `bins` array. This allows us to generate a *raster plot*.

In [ ]:
hom_spike_i = np.nonzero(hom_spikes)[0]
hom_spike_t = bins[hom_spike_i]
# here's one way to plot a raster of spike times
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 3))
axes.plot(hom_spike_t, np.zeros_like(hom_spike_t), "|")

### More spike train statistics

**Q** Here's a slightly harder question about the properties of Poisson processes. Calculate the interspike *intervals* from `spike_t` (hint: look at the documentation for `np.diff`), then plot a histogram. What function does this look like? Calculate the sample mean and variance. How do these relate to the rate of the process? What is the coefficient of variation of the interspike distribution (CV = $\mu / \sigma$)?

### Inhomogeneous Poisson Processes

It's also possible for the rate of a Poisson process to vary in time; that is, for $\lambda$ to be a function of $t$.

$$p(n|\lambda(t)) = \frac{\lambda(t)^n}{n!}\exp(-\lambda(t))$$

As before, we need to discretize time and determine the probability that there is a spike in some interval $(t, t + \Delta)$; the only difference is that some intervals are more likely to have spikes than others.

We could simulate an inhomogeneous Poisson process in much the same way as we did above, but we need to vary $\lambda$ in each bin.

Let's look at a simple example where the intensity linearly ramps up from zero and then back down.

In [ ]:
np.random.seed(1)
T     = 100     # s
Delta = 0.001   # s
N     = int(T / Delta)
bins  = np.arange(0, T, Delta)
# rate is now a function of time
inh_rate  = np.concatenate([np.linspace(0.0, 4.0, N//2),
                            np.linspace(4.0, 0.0, N//2)])

# generate N values from a uniform distribution
rand = dists.uniform().rvs(N)
# this is an alternative method of simulating spiking based on the Bernoulli distribution
# compare each value to lambda = rate * Delta; if it's greater, then the bin gets a spike
lam  = inh_rate * Delta
inh_spikes = (inh_rate * Delta) > rand
inh_spike_i = np.nonzero(inh_spikes)[0]
inh_spike_t = bins[inh_spike_i]
# here's one way to plot a raster of spike times
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(9, 3))
axes.plot(bins, inh_rate)
axes.plot(inh_spike_t, np.zeros_like(inh_spike_t), "k|")


**Q** Calculate the mean, variance, and CV of the interspike intervals for this spike train (`inh_spike_t`). Is the relationship with the rate the same as you discovered for the homogeneous process?

## Estimating spike rates

Let's think about how we can estimate $\lambda$ for Poisson processes.

If we assume that the process is homogeneous over each trial, then we have a simple observational model where the number of spikes is a random sample from the Poisson distribution.

$$p(y_i|\lambda) = \frac{\lambda^n}{n!}\exp(-\lambda)$$

Given a set of trials, we can estimate $\lambda$ from the sample mean of the spike count:

$$\hat{\lambda} = \sum_i y_i$$

**Q**: What is the estimated intensity of the homogeneous spike train we generated above (`hom_spike_t`)?

The problem is a lot trickier if the process is inhomogeneous, because now we're trying to estimate a continuous function of time, $\lambda(t)$.

In this setting, people usually talk about rate rather than intensity ($\lambda$), so we'll use $r(t)$ from here on.

The issue we confront is that $r(t)$ is a continuous function. We can discretize it into small intervals of $(t, t + \Delta)$ and count the number of spike in each interval, but as we make $\Delta$ smaller to get higher temporal resolution, we reach the point at which each bin has either one or zero spikes, which doesn't tell us much about the rate. We can address this problem by averaging across multiple trials. If we use $\langle \rangle$ to denote averaging across trials, this looks like:

$$r(t) = \frac{1}{\Delta} \int_t^{t+\Delta} d\tau\; \langle \rho(t) \rangle$$

You hopefully can see that as $\Delta$ gets smaller, the number of trials you need to average to get a smooth function gets larger. So part of our problem is to determine what $\Delta$ should be. More practically, at what time scale do we think the rate is changing?

There are a number of different ways of approximating $r(t)$. We'll look at a couple.

### Spike time histogram

For historical reasons, this is also called a peri-stimulus spike time histogram (PSTH), even when there isn't a stimulus.

The simplest way of approximating the rate is to divide the interval up into a fixed number of bins of duration $\Delta$ and count how many spikes occurred in each bin. The rate is simply the number of spikes divided by $\Delta$.

The main problem with histograms is that setting the bin size is largely subjective. Try adjusting the bin count (which is just the inverse of the bin size) variable and see what gives you the best tradeoff between variability and temporal resolution.

There is still active development of new methods for adaptively setting bin sizes in timing histograms.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 3))
bin_size = 5.0
bin_count = widgets.IntSlider(value=int(T / bin_size), min=1, max=100, step=1, 
                              description="bin count:", continuous_update=True)
display(bin_count)

ax.plot(bins, inh_rate)
ax.plot(inh_spike_t, np.zeros_like(inh_spike_t), "k|")
r_est, edges  = np.histogram(inh_spike_t, bins=np.arange(0, T + bin_size, bin_size))
p = ax.step(edges[1:], r_est / bin_size)

def update(bin_count):
    bin_size = T / bin_count
    r_est, edges  = np.histogram(inh_spike_t, bins=np.arange(0, T + bin_size, bin_size))
    print("bin size:", bin_size)
    p[0].set_data(edges[1:], r_est / bin_size)
    
widgets.interactive_output(update, {"bin_count": bin_count})

### Smoothing

Another problem with the PSTH is that the count in each bin can depend very strongly on where the edges of the bins are.

One solution to this problem is to use a **sliding window**. The simplest window is simply a square function with a defined width and a total area equal to 1.0.

For example, here's a 10 ms window.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3, 3))
window_size = 2.0
w_t = np.arange(-10.0, 10.0, Delta)
w = np.zeros_like(w_t)
w[(-window_size/2 < w_t) & (w_t <window_size/2)] = 1. / window_size
ax.plot(w_t, w)

### Convolution

We can express the sliding window operation as a sum of the window function for the values of the spike times.

$$r(t) \approx \sum_{i=0}^{N-1} w(t - t_i)$$

This is equivalent to doing an integral over the response function:

$$r(t) \approx \int_{-\infty}^{\infty} d\tau\; w(\tau) \rho(t - \tau)$$

This integral is also called a linear **convolution** or filter, and we'll be seeing a lot of them.

Numpy has a function that can calculate this convolution:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 3))
r_est = np.convolve(inh_spikes, w, mode='same')
ax.plot(bins, inh_rate)
ax.plot(inh_spike_t, np.zeros_like(inh_spike_t), "k|")
ax.plot(bins, r_est)

You can use any function as a window as long as it goes to zero outside $\tau = 0$ and its integral is 1.0.

A popular choice is to use a Gaussian, which smooths the function by downweighting points further away from $\tau = 0$

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3, 3))
sigma = 2.0
w = 1 / np.sqrt(2 * np.pi) / sigma * np.exp(-w_t**2 / 2 / sigma**2)
plt.plot(w_t, w)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 3))
sigma_w = widgets.FloatSlider(value=sigma, min=0.1, max=5.0, step=0.1,
                              description="sigma:", continuous_update=False)
display(sigma_w)

ax.plot(bins, inh_rate)
ax.plot(inh_spike_t, np.zeros_like(inh_spike_t), "k|")
r_est = np.convolve(inh_spikes, w, mode='same')
p = ax.plot(bins, r_est)

def update(sigma):
    w_T = 10
    w_t = np.arange(-w_T, w_T, Delta)
    w = 1 / np.sqrt(2 * np.pi) / sigma * np.exp(-w_t**2 / 2 / sigma**2)
    r_est = np.convolve(inh_spikes, w, mode='same')
    p[0].set_data(bins, r_est)
    
widgets.interactive_output(update, {"sigma": sigma_w})

### Averaging trials

Hopefully, these exercises have illustrated the fundamental tradeoff between variance and temporal resolution. As you increase the bin (or window) size ($\Delta$), the estimated rate becomes less variable, but the temporal resolution decreases. Thus, smoothing can interfere with detecting rapid changes in the underlying rate function.

As noted above, the solution to this problem is to average across trials. In essence, this gives you multiple independent estimates of the rate at any given instant, thereby reducing the amount of smoothing you need.

We can represent spiking responses over multiple trials using a two-dimensional array. Here's a simulation of 10 trials to the above rate function:

In [ ]:
# we're keeping all the parameters the same as above, but resetting the random seed
np.random.seed(1)
n_trials = 10

# it's often a good idea to initialize the array first, then fill it up
# recall that N is the number of bins
mt_spikes = np.zeros((N, n_trials))
# we're going to store the spike times in a ragged array (i.e. a list of arrays)
mt_spikes_t = []
for trial in range(n_trials):
    # generate N values from a uniform distribution
    rand = dists.uniform().rvs(N)
    lam  = inh_rate * Delta
    spikes = (inh_rate * Delta) > rand
    idx = np.nonzero(spikes)[0]
    mt_spikes[:, trial] = spikes
    mt_spikes_t.append(bins[idx])

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(9, 3))
axes.plot(bins, inh_rate)    
for i, trial in enumerate(mt_spikes_t):
    axes.plot(trial, np.ones_like(trial) * i / n_trials, "k|")

The rate function is calculated by averaging smoothed estimates across trials. Compare the plot below to the smoothed estimate based on 1 trial.

In [ ]:
sigma = 2.0
w_T = 10
w_t = np.arange(-w_T, w_T, Delta)
w = 1 / np.sqrt(2 * np.pi) / sigma * np.exp(-w_t**2 / 2 / sigma**2)
r_est = np.convolve(mt_spikes.mean(1), w, mode='same')

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(9, 3))
axes.plot(bins, inh_rate)
axes.plot(bins, r_est)

### Putting it together

For the rest of the class period, work on the following exercises.

#### Exercise 1

Consider an inhomogeneous Poisson process with a time-varying rate specified by:

$$r(t) = r_\mathrm{max}[\sin(2\pi\omega t) + 1]$$

Use $\omega$ = 2.1 Hz, $r_\mathrm{max} = 50$ Hz, and a response interval from 0 to 2 s.

**Q** Plot r(t) with a time step of 1 ms (0.001 s)

**Q** Generate 20 independently simulated spike trains and plot them as rasters. There is code in previous notebooks you can use to make the raster plot.

**Q**. Using a bin size of 10 ms, calculate the PSTHs averaged from the first 10 trials and the last 10 trials.

**Q** Now simulate 1000 trials and calculate a PSTH from the first and second half. How do these PSTHs relate to $\lambda(t)$? Do more trials give you a more precise estimate of the rate?

**Q** Calculate a PSTH *for each trial* of the 1000-trial simulation. This will yield a 1000 x 200 array, with trials along one axis and time along the other. For each time bin, calculate and plot the mean, variance, and Fano factor. Each of these will be a 200-element array.

**Q** Repeat the analysis in the previous question but with a bin size of 150 ms. What happens to the Fano factor? Why is it important to choose a bin size such that the rate is not changing much within each bin?